# 自动求导

假设对函数$y = 2x^{T}x$，关于列向量 $x$ 求导，实际上就是 $y = 2x^2$（符号形式）

In [2]:
import torch
x = torch.arange(4.0)
x

tensor([0., 1., 2., 3.])

在计算 $y$ 关于 $x$ 的梯度之前，需要一个地方来存储梯度

In [8]:
x.requires_grad_(True) # 等价 x = torch.arange(4.0, requires_grad = True)
x.grad # 默认为 None

计算 $y$, 隐式计算图

In [11]:
y = 2 * torch.dot(x, x)
y

tensor(28., grad_fn=<MulBackward0>)

通过反向传播函数 `backward()` 自动计算 $y$ 关于 $x$ 每个分量的梯度

In [12]:
y.backward()
x.grad

tensor([ 0.,  4.,  8., 12.])

In [13]:
x.grad == 4 * x

tensor([True, True, True, True])

默认情况下，`Pytorch` 会累积梯度，在计算下一个值之前要先清除之前值

In [27]:
x.grad.zero_()
y = x.sum() # x1 + x2 + x3 + x4
y.backward()
x.grad

tensor([1., 1., 1., 1.])

深度学习中，计算微分矩阵不是目的，目的是批量中每个样本单独计算的偏导数之和

In [29]:
x.grad.zero_()
y = x * x
y.sum().backward()
x.grad

tensor([0., 2., 4., 6.])

将某些计算移到计算图之外，使用 `detach()` 函数将 $u$ 作为一个常数，而不是关于 $x$ 的函数

In [37]:
x.grad.zero_()
y = x * x
u = y.detach()
z = u * x
z.sum().backward()
x.grad, x.grad == u

(tensor([0., 1., 4., 9.]), tensor([True, True, True, True]))

即使构建函数的计算图需要通过 Python 控制流，我们仍然可以计算得到的变量的梯度

In [53]:
def f(a):
    b = a * 2
    while b.norm() < 1000:
        b = b * 2
    if b.sum() > 0:
        c = b
    else:
        c = 100 * b
    return c

a = torch.randn(size = (), requires_grad = True)
d = f(a)
d.backward()
a.grad == d / a # f(a) 最终是关于 a 的线性函数，b = f(a)，b 对 a 求导最终就是线性函数的系数，也就是 b / a

tensor(True)